In [1]:
import pickle
import collections
import json
import requests

In [2]:
meta_path = "../meta_data/" 

In [3]:
def fetch_ethnicity(f_name, l_name):
    ethnicity = "UNKNOWN"
    try:
        response = requests.get(
            "http://abel.lis.illinois.edu/cgi-bin/ethnea/search.py?Fname=" + f_name + "&Lname=" + l_name + "&format=json")
        response = response.text
        response = response.replace("'", "\"")
        json_response = json.loads(response)
        ethnicity = json_response['Ethnea']
    except:
        print("Failed for :",f_name," ",l_name)
    return ethnicity

In [4]:
with open(meta_path + 'atomic_names_list.pickle', 'rb') as handle:
    atomic_names_list = pickle.load(handle)

atomic_name_ethnicity_dict = {}

for atomic_name in atomic_names_list:
    f_name,l_name = (atomic_name.rsplit('_', 1)[0], atomic_name.rsplit('_', 1)[1]) if (len(atomic_name.rsplit('_', 1)) == 2) else (atomic_name,atomic_name)
    atomic_name_eth = fetch_ethnicity(f_name, l_name)
    atomic_name_ethnicity_dict[atomic_name] = atomic_name_eth
    
with open(meta_path + 'ethnicities.pickle', 'wb') as handle:
    pickle.dump(atomic_name_ethnicity_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
ethnicity_counts = collections.Counter(atomic_name_ethnicity_dict.values())
with open(meta_path + 'ethnicity_counts.pickle', 'wb') as handle:
    pickle.dump(ethnicity_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)